## Recommender env: design

In [1]:
# HIDDEN
import gym

In [2]:
# TODO: adapt images from Sven for this

#### Nutzerverhalten simulieren

- Simuliere das Nutzerverhalten, wenn du **wiederholt** auf Produktempfehlungen reagierst.
- Das wichtigste zu simulierende Verhalten:

&gt; Das Empfehlen von "Junk Food"-Produkten ist kurzfristig gut, aber auf lange Sicht schlecht.

- Das ist allein unsere Entscheidung als Designer der Environment 
- Vielleicht möchtest du eine andere Art von Nutzerverhalten simulieren/erfassen.
  - Oder du kannst aus den Daten zum Nutzerverhalten lernen - dazu später mehr!
- Aber das wird unser Beispiel für den Moment sein.

#### Schokolade

- Wir werden jeden Artikel mit einem "Süßigkeitsgrad" modellieren 
- Artikel mit hohem Süßigkeitsgrad bezeichnen wir als "Süßigkeiten".

![](img/candy.jpg)

- Dabei kann es sich um kurze, alberne Videos oder billigen Schnickschnack im Angebot handeln.
- Kurzfristig lieben die Nutzer/innen Süßigkeiten, aber zu viel Schokolade führt langfristig zu Unzufriedenheit.

#### Gemüsesorten

- Auf der anderen Seite bezeichnen wir süßstoffarme Produkte als "Gemüse".

![](img/veggies.jpg)

- Das können lehrreiche Dokumentationen sein oder langweilige, aber nützliche Dinge usw.
- Kurzfristig macht das Gemüse den Nutzern keinen Spaß, aber langfristig steigert es die Zufriedenheit.

#### Zuckerspiegel

- Wir stellen uns vor, dass unsere Nutzer einen **Zuckerspiegel** haben, der angibt, wie viele Süßigkeiten sie in letzter Zeit gegessen haben.
- Der Zuckerspiegel des Benutzers (oder die Vorstellung von einem Zuckerspiegel) _ist dem Agenten nicht bekannt_!
- Aber im Moment entwerfen wir den Simulator, damit wir alles wissen.

#### Dynamik des Zuckerspiegels

- Wir müssen entscheiden, wie sich der Zuckerspiegel mit dem Verzehr der Artikel verändert.
- Ein einfacher Ansatz ist:

&gt; Wenn ein Gegenstand verzehrt wird, bewegt sich der Zuckergehalt in Richtung der Süße des Gegenstands.

Beispiele:

- Wenn dein Zuckerspiegel 0,2 ist und du einen Gegenstand mit der Süße 0,5 konsumierst, steigt dein Zuckerspiegel ⬆️
- Wenn dein Zuckerspiegel 0,2 ist und du einen Gegenstand mit der Süße 0,1 konsumierst, sinkt dein Zuckerspiegel ⬇️

#### Dynamik des Zuckerspiegels

- Wie können wir das mathematisch darstellen?
- Wir können dies versuchen:

&gt; neuer Zuckerspiegel = ⍺ (alter Zuckerspiegel) + (1 - ⍺) (Artikelsüße)

- Dabei ist ⍺ eine Zahl zwischen 0 und 1, die bestimmt, wie "hartnäckig" der Zuckergehalt ist.

In [3]:
# HIDDEN 
# note the slide above and below are partially the same - just want to hide the bottom half at first

#### Dynamik des Zuckerspiegels

- Wie können wir das mathematisch darstellen?
- Wir können dies versuchen:

&gt; neuer Zuckerspiegel = ⍺ (alter Zuckerspiegel) + (1 - ⍺) (Artikelsüße)

- Dabei ist ⍺ eine Zahl zwischen 0 und 1, die bestimmt, wie "hartnäckig" der Zuckergehalt ist.
- Wenn zum Beispiel ⍺=1 ist, wird die obige Gleichung zu

&gt; neuer Zuckerspiegel = alter Zuckerspiegel

und der Zuckerspiegel ändert sich nie. Wenn ⍺=0 ist, haben wir

&gt; neuer Zuckerspiegel = Gegenstandssüße

was bedeutet, dass ein einziger Gegenstand den Zuckerspiegel des Nutzers vollständig verändern kann.

- Für ⍺ zwischen 0 und 1 haben wir eine Kombination aus dem alten Zuckerspiegel und der Item-Süße.

#### Dynamik des Zuckerspiegels

Mit dieser Funktion können wir das obige umsetzen:

In [4]:
def update_sugar_level(sugar_level, item_sweetness, alpha=0.9):
    return alpha * sugar_level + (1 - alpha) * item_sweetness

Probieren wir es aus, um sicherzustellen, dass das Verhalten sinnvoll ist (mit dem Standardwert ⍺=0,9):

In [5]:
sugar_level = 0.2
sugar_level = update_sugar_level(sugar_level, 0.8)
sugar_level

0.26

Der Artikel war süß (0,8), so dass der Zuckerspiegel ziemlich in die Höhe ging.

In [6]:
sugar_level = update_sugar_level(sugar_level, 0.3)
sugar_level

0.264

Der Artikel Süße lag etwas über dem Zuckerspiegel, so dass der Zuckerspiegel leicht anstieg.

#### Dynamik des Zuckerspiegels

In [7]:
sugar_level = update_sugar_level(sugar_level, 0.01)
sugar_level

0.2386

Der Artikel war nicht süß, also ging der Zuckergehalt runter.

#### Wirkung von alpha

Wir können sehen, dass sich der Zuckerspiegel mit einem kleineren Alpha viel schneller ändert:

In [8]:
sugar_level = update_sugar_level(sugar_level, 0.0, alpha=0.5)
sugar_level

0.1193

#### Belohnung

- Na toll, jetzt haben wir die Dynamik des Zuckerspiegels im Griff!
- Das zweite wichtige Teil des Puzzles ist die Belohnung.
- Was wir wollen:

1. Je süßer der Gegenstand, desto höher die Belohnung (lecker, Süßigkeiten!)
2. Ein höherer Zuckergehalt führt zu einer geringeren Belohnung (ach, zu viel Süßes!)

Eine einfache Möglichkeit, diese Effekte zu kombinieren, ist, sie miteinander zu multiplizieren:

&gt; Belohnung = Süße des Gegenstands * (1 - Zuckergehalt)

#### Implementierung von Belohnungen

&gt; Belohnung = Gegenstand Süße * (1 - Zuckergehalt)

Wir können dies wie folgt kodieren:

In [9]:
def reward(sugar_level, item_sweetness):
    return item_sweetness * (1 - sugar_level)

Wir werden diese Teile im nächsten Abschnitt verwenden, wenn wir unsere Environment implementieren!

#### Beobachtungsraum

- Als Nächstes müssen wir die Beobachtungen einrichten 
- Unsere Beobachtungen sind die _Merkmale der Artikelkandidaten_.
- Der Einfachheit halber nehmen wir an, dass es nur 1 Merkmal gibt, nämlich die Süße des Artikels.
- Der Agent sieht also eine Reihe von Süßigkeitsstufen und wählt eine davon aus.

#### Aktionsraum

- In diesem Umfeld ist die Aktion das ausgewählte Element, das angesichts der Kandidaten empfohlen wird.

#### Lass uns das Gelernte anwenden!

## Big-Picture
<!-- multiple choice -->

Welche der folgenden Aussagen über die simulierte Empfehlungs-RL-Environment, die wir erstellen, ist **NICHT** wahr?

- [ ] Die Environment enthält ein stark vereinfachtes Modell des Nutzerverhaltens, aber ein trainierter Agent kann trotzdem nützlich sein, um Empfehlungen zu geben.
- [x] Die Environment gibt genau wieder, wie sich reale Nutzer verhalten.
- [ ] Die Environment ist ein guter Ausgangspunkt, und wir möchten die Komplexität im Laufe unserer Arbeit vielleicht noch erhöhen.
- [Die Environment berücksichtigt die Tatsache, dass die Nutzer/innen auf verschiedene Gegenstände unterschiedlich reagieren und dass diese Reaktionen von ihrer Vergangenheit abhängen können.

## Empfehlende Belohnungen
<!-- multiple choice -->

Erinnern Sie sich, dass unsere Belohnungsfunktion lautet

&gt; Belohnung = Süße des Artikels * (1 - Zuckergehalt)

#### Kurzfristige Zufriedenheit

Richtig oder falsch: Zu einem bestimmten Zeitpunkt ist die _unmittelbare_ Belohnung für Süßigkeiten _immer_ größer als für Gemüse.

- [Richtig | Die unmittelbare Belohnung ist direkt proportional zur Süße des Gegenstands.
- [Falsch | Sieh dir die Formel oben genauer an!

#### Langfristige Zufriedenheit

Richtig oder falsch: Zu einem bestimmten Zeitpunkt ist die _langfristige Gesamtbelohnung_ für die Empfehlung von Gemüse _immer_ größer als für Süßigkeiten.

- [Richtig | Es ist kompliziert zu bestimmen, was langfristig das Beste ist - das muss unser Agent lernen!
- [x] Falsch

## Zucker-Crash
<!-- coding exercise -->

Nehmen wir an, dein Zuckerspiegel beginnt bei 0,5 und du hast bei jedem Schritt nur zwei Produkte zur Auswahl: Mega-Gemüse (Süße = 0) und Mega-Süßigkeiten (Süße = 1). Du gibst 3 Empfehlungen hintereinander ab, wobei Alpha = 0,7 gilt. Nutze das Kodierfenster unten, um mit verschiedenen Optionen zu spielen und die beste Reihenfolge der Empfehlungen im Hinblick auf die _gesamte_ Belohnung zu finden 

In [10]:
# EXERCISE

def update_sugar_level(sugar_level, item_sweetness, alpha=0.9):
    return alpha * sugar_level + (1 - alpha) * item_sweetness

def reward(sugar_level, item_sweetness):
    return item_sweetness * (1 - sugar_level)

# MODIFY THIS LIST
# But make sure it always contains 3 items, each 0 or 1
recommendations = [0, 0, 0]

# starting sugar level
sugar_level = 0.5

total_reward = 0

for item_sweetness in recommendations:
    
    # add reward
    immediate_reward = reward(sugar_level, item_sweetness)
    total_reward += immediate_reward
    
    # update sugar level
    sugar_level = update_sugar_level(sugar_level, item_sweetness, alpha=0.7)
    
    print(f"  Received reward {immediate_reward:.5f}, new sugar level {sugar_level:.5f}")
    
print("Total reward after 5 recommendations:", total_reward)

  Received reward 0.00000, new sugar level 0.35000
  Received reward 0.00000, new sugar level 0.24500
  Received reward 0.00000, new sugar level 0.17150
Total reward after 5 recommendations: 0.0


In [11]:
# SOLUTION

def update_sugar_level(sugar_level, item_sweetness, alpha=0.9):
    return alpha * sugar_level + (1 - alpha) * item_sweetness

def reward(sugar_level, item_sweetness):
    return item_sweetness * (1 - sugar_level)

# MODIFY THIS LIST
# But make sure it always contains 3 items, each 0 or 1
recommendations = [0,1,1]

# starting sugar level
sugar_level = 0.5

total_reward = 0

for item_sweetness in recommendations:
    
    # add reward
    immediate_reward = reward(sugar_level, item_sweetness)
    total_reward += immediate_reward
    
    # update sugar level
    sugar_level = update_sugar_level(sugar_level, item_sweetness, alpha=0.7)
    
    print(f"  Received reward {immediate_reward:.5f}, new sugar level {sugar_level:.5f}")
    
print("Total reward after 5 recommendations", total_reward)

  Received reward 0.00000, new sugar level 0.35000
  Received reward 0.65000, new sugar level 0.54500
  Received reward 0.45500, new sugar level 0.68150
Total reward after 5 recommendations 1.105


#### Was war die beste Policy in diesem Beispiel?

- [x] 1 Gemüse, um den Zuckerspiegel zu senken, dann 2 Süßigkeiten für die süße, süße Belohnung.
- [x] Süßigkeiten, dann Gemüse für die Gesundheit, dann wieder Süßigkeiten.
- [ ] Gemüse für alle Fälle!
- [ ] Süßigkeiten bis zum Schluss!